In [11]:
import torch
import torchvision.models as models
import pandas as pd
import torch.nn as nn
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
import torch.optim as optim
import numpy as np
from sklearn.metrics import f1_score
from PIL import Image
import os

In [12]:
model = models.resnet50()
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [13]:
n_classes = 2
model.fc = nn.Linear(model.fc.in_features, n_classes)

In [14]:
train_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomRotation(30),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.GaussianBlur(kernel_size=5, sigma=(0.1,2.0)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

In [15]:
test_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

In [16]:
train_data = datasets.ImageFolder(root=f'../data/train_data', transform=train_transform)
val_data = datasets.ImageFolder(root=f'../data/val_data', transform=test_transform)

In [17]:
train_loader = DataLoader(dataset=train_data, batch_size=64, shuffle=True)
val_loader = DataLoader(dataset=val_data, batch_size=64, shuffle=True)

In [18]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [19]:
for param in model.parameters():
    param.requires_grad = False

for param in model.fc.parameters():
    param.requires_grad = True

In [20]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=model.parameters(), lr=0.0001)
scheduler = optim.lr_scheduler.StepLR(optimizer=optimizer, step_size=5, gamma=0.1)

In [21]:
num_epoches = 15
for epoch in range(num_epoches):
    model.train()
    running_loss = 0.0
    
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss
    scheduler.step()
    print(f'epoch {epoch + 1}, Loss: {running_loss / len(train_loader):.4f}')
torch.save(model.state_dict(), '../model/ResNet_1.pth')

epoch 1, Loss: 0.5133
epoch 2, Loss: 0.4738
epoch 3, Loss: 0.4541
epoch 4, Loss: 0.4462
epoch 5, Loss: 0.4407
epoch 6, Loss: 0.4329
epoch 7, Loss: 0.4300
epoch 8, Loss: 0.4303
epoch 9, Loss: 0.4300
epoch 10, Loss: 0.4281
epoch 11, Loss: 0.4306
epoch 12, Loss: 0.4293
epoch 13, Loss: 0.4294
epoch 14, Loss: 0.4269
epoch 15, Loss: 0.4263


In [22]:
model.load_state_dict(torch.load('../model/ResNet_1.pth'))

<All keys matched successfully>

In [23]:
model.eval()
val_y_true, val_y_pred = [], []
with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        val_y_true.extend(labels.tolist())
        val_y_pred.extend(preds.tolist())

In [24]:
f1_score(y_true=val_y_true, y_pred=val_y_pred)

0.7806637806637805

In [25]:
test_csv = pd.read_csv('../data/test.csv')
image_list = list(test_csv['id'])

In [26]:
test_y_pred = []
model.eval()
for image_id in image_list:
    image_path = os.path.join('../data', image_id)
    image = Image.open(image_path).convert('RGB')
    
    image = test_transform(image)
    image = image.unsqueeze(0)

    image = image.to(device)
    model.to(device)
    with torch.no_grad():
        output = model(image)
    _,pred = torch.max(output, 1)
    test_y_pred.extend(pred.tolist())

In [27]:
test_csv['label'] = test_y_pred

In [32]:
test_csv.to_csv('../data/result/ResNet_pred_1.csv', index=False)